# 🌴 Asistente Turístico de Tenerife - RAG Demo

## 1. Configuración del Entorno
Cargamos las librerías necesarias y verificamos la API Key.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ Key cargada: {api_key[:5]}...")
else:
    print("❌ Error de Key")